# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [38]:
import pandas as pd
from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys

navegador = wd.Chrome() #definindo o webdriver

# Passo 1: Buscar as cotações

# Dólar
navegador.get("https://www.google.com/") # abre o webdriver
navegador.find_element('xpath',
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("dolar hoje") # seleciona o campo de busca do google e busca o dolar
navegador.find_element('xpath',
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER) # busca
cotacao_dolar = navegador.find_element('xpath','//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

# Euro 
navegador.get("https://www.google.com/") # abre o webdriver
navegador.find_element('xpath',
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("euro hoje") # seleciona o campo de busca do google e busca o euro
navegador.find_element('xpath',
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER) # busca
cotacao_euro = navegador.find_element('xpath','//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

# Ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje") # entra no site do ouro
cotacao_ouro = navegador.find_element('xpath','//*[@id="comercial"]').get_attribute('value') #pega o valor do ouro
cotacao_ouro = cotacao_ouro.replace(",",".")
navegador.quit()

print (cotacao_dolar)
print (cotacao_euro)
print (cotacao_ouro)


5.2442
5.3412
292.78


### Agora vamos atualizar a nossa base de preços com as novas cotações

- Importando a base de dados

In [39]:
# Passo 2: Atualizar a base de dados
tabela = pd.read_excel("Produtos.xlsx")
display (tabela)
print (tabela.info())

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Produtos         7 non-null      object 
 1   Preço Original   7 non-null      float64
 2   Moeda            7 non-null      object 
 3   Cotação          7 non-null      int64  
 4   Preço de Compra  7 non-null      float64
 5   Margem           7 non-null      float64
 6   Preço de Venda   7 non-null      float64
dtypes: float64(4), int64(1), object(2)
memory usage: 520.0+ bytes
None


- Atualizando os preços e o cálculo do Preço Final

In [40]:
# Passo 3: Recalcular os preços

# Atualizar as cotações

tabela.loc[tabela["Moeda"]=="Dólar","Cotação"] = float(cotacao_dolar)
tabela.loc[tabela["Moeda"]=="Euro","Cotação"] = float(cotacao_euro)
tabela.loc[tabela["Moeda"]=="Ouro","Cotação"] = float(cotacao_ouro)
display(tabela)
# Recalcular preço de compra (original x cotação)
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]
# Recalcular preço de venda (preço de compra x margem)
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.2442,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,5.3412,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5.2442,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5.2442,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,5.3412,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5.2442,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,292.7800,7000.00,1.15,8050.000


### Agora vamos exportar a nova base de preços atualizada

In [41]:
# Passo 4: Exportar a base atualizada

tabela.to_excel("Produtos Novo.xlsx", index=False)

In [42]:
tabela2 = pd.read_excel("Produtos Novo.xlsx")
display(tabela2)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.2442,5244.147558,1.40,7341.806581
1,Carro Renault,4500.00,Euro,5.3412,24035.400000,2.00,48070.800000
2,Notebook Dell,899.99,Dólar,5.2442,4719.727558,1.70,8023.536849
3,IPhone,799.00,Dólar,5.2442,4190.115800,1.70,7123.196860
4,Carro Fiat,3000.00,Euro,5.3412,16023.600000,1.90,30444.840000
5,Celular Xiaomi,480.48,Dólar,5.2442,2519.733216,2.00,5039.466432
6,Joia 20g,20.00,Ouro,292.7800,5855.600000,1.15,6733.940000
